# MQTT SUBSCRIBE 測試應用程式

這是一個測試用的 MQTT 訂閱應用程式，用於接收感測器數據。

## 功能
- 連接到本地 MQTT Broker
- 訂閱指定主題
- 即時顯示接收到的訊息
- 解析 JSON 格式的數據
- 顯示訊息統計和分析

## 使用說明
1. **按順序執行 Cell 1 → 2 → 3 → 4**
2. Cell 4（建立連線並訂閱）只執行一次
3. Cell 5（查看統計）可以重複執行
4. 完成後執行 Cell 6 清理連線

## 測試方式
- 先啟動此訂閱者（lesson6_2.ipynb）
- 再執行發布者（lesson6_1.ipynb）發送訊息
- 即可看到即時接收的數據


In [1]:
# Cell 1: 匯入必要的套件
import paho.mqtt.client as mqtt
import json
import time
from datetime import datetime

print("✅ 套件匯入成功！")


✅ 套件匯入成功！


In [2]:
# Cell 2: MQTT 配置

# MQTT Broker 位址設定
BROKER = "localhost"  # 本機 Broker
# BROKER = "192.168.1.16"  # 或使用 Raspberry Pi 的 IP

PORT = 1883
TOPIC = "客廳/感測器"  # 訂閱的主題
# TOPIC = "#"  # 訂閱所有主題

# 客戶端 ID（加入時間戳記以避免重複）
CLIENT_ID = f"jupyter_subscriber_{int(time.time())}"

# 訊息統計變數
message_count = 0
message_history = []

print("=" * 60)
print("📡 MQTT 訂閱設定")
print("=" * 60)
print(f"   Broker: {BROKER}:{PORT}")
print(f"   Topic: {TOPIC}")
print(f"   Client ID: {CLIENT_ID}")
print("=" * 60)


📡 MQTT 訂閱設定
   Broker: localhost:1883
   Topic: 客廳/感測器
   Client ID: jupyter_subscriber_1765561390


In [3]:
# Cell 3: 定義回調函數

connected = False

def on_connect(client, userdata, flags, reason_code, properties):
    """連線成功時的回調"""
    global connected
    if reason_code == 0:
        print("✅ 成功連接到 MQTT Broker")
        # 連線成功後自動訂閱主題
        client.subscribe(TOPIC, qos=1)
        print(f"📬 已訂閱主題: {TOPIC}")
        connected = True
    else:
        print(f"❌ 連線失敗，錯誤碼: {reason_code}")
        connected = False

def on_message(client, userdata, msg):
    """接收到訊息時的回調"""
    global message_count, message_history
    
    message_count += 1
    
    try:
        # 嘗試解析 JSON 格式的訊息
        payload = msg.payload.decode('utf-8')
        data = json.loads(payload)
        
        # 顯示訊息內容
        print(f"\n{'='*60}")
        print(f"📩 收到訊息 #{message_count}")
        print(f"{'='*60}")
        print(f"   主題: {msg.topic}")
        print(f"   時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
        # 顯示數據內容
        if isinstance(data, dict):
            if 'temperature' in data:
                print(f"   🌡️  溫度: {data['temperature']}°C")
            if 'humidity' in data:
                print(f"   💧 濕度: {data['humidity']}%")
            if 'light_status' in data:
                print(f"   💡 電燈: {data['light_status']}")
            if 'device' in data:
                print(f"   📱 裝置: {data['device']}")
            if 'timestamp' in data:
                print(f"   ⏰ 發送時間: {data['timestamp']}")
        else:
            print(f"   數據: {data}")
        
        # 儲存到歷史記錄（最多保留最近 50 筆）
        message_history.append({
            'count': message_count,
            'topic': msg.topic,
            'data': data,
            'received_at': datetime.now().isoformat()
        })
        
        if len(message_history) > 50:
            message_history.pop(0)
            
    except json.JSONDecodeError:
        # 如果不是 JSON 格式，直接顯示原始內容
        print(f"\n{'='*60}")
        print(f"📩 收到訊息 #{message_count}")
        print(f"{'='*60}")
        print(f"   主題: {msg.topic}")
        print(f"   時間: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"   內容: {msg.payload.decode('utf-8')}")
        
    except Exception as e:
        print(f"❌ 處理訊息時發生錯誤: {e}")

def on_subscribe(client, userdata, mid, reason_code_list, properties):
    """訂閱成功時的回調"""
    print(f"✅ 訂閱成功 (ID: {mid})")

def on_disconnect(client, userdata, flags, reason_code, properties):
    """斷線時的回調"""
    global connected
    connected = False
    if reason_code == 0:
        print("👋 已正常斷開連線")
    else:
        print(f"⚠️ 意外斷線，錯誤碼: {reason_code}")

print("✅ 回調函數定義完成")


✅ 回調函數定義完成


In [ ]:
# Cell 4: 建立連線並訂閱（⚠️ 只執行一次！）

# 檢查是否已有連線
if 'client' in globals():
    print("⚠️ 警告：client 已存在！")
    print("   如需重新連線，請先執行 Cell 6 清理，然後重啟 Kernel")
else:
    # 重置統計
    message_count = 0
    message_history = []
    
    try:
        # 建立新客戶端
        client = mqtt.Client(
            callback_api_version=mqtt.CallbackAPIVersion.VERSION2,
            client_id=CLIENT_ID,
            clean_session=True
        )
        
        # 設定回調函數
        client.on_connect = on_connect
        client.on_message = on_message
        client.on_subscribe = on_subscribe
        client.on_disconnect = on_disconnect
        
        # 連線
        print(f"🔌 正在連接到 {BROKER}:{PORT}...")
        client.connect(BROKER, PORT, 60)
        
        # 啟動網路迴圈（非阻塞模式）
        client.loop_start()
        
        # 等待連線建立
        for i in range(50):  # 最多等 5 秒
            if connected:
                break
            time.sleep(0.1)
        
        if connected:
            print("\n" + "=" * 60)
            print("✅ MQTT 訂閱者已就緒，等待訊息...")
            print("=" * 60)
            print("💡 提示：")
            print("   1. 此訂閱者會持續接收訊息")
            print("   2. 可以執行 lesson6_1.ipynb 來發布測試訊息")
            print("   3. 執行 Cell 5 查看統計資訊")
            print("   4. 執行 Cell 6 關閉連線")
            print("=" * 60)
        else:
            print("\n❌ 連線失敗或超時")
            
    except Exception as e:
        print(f"❌ 連線失敗: {e}")
        import traceback
        traceback.print_exc()


🔌 正在連接到 localhost:1883...
✅ 成功連接到 MQTT Broker
📬 已訂閱主題: 客廳/感測器
✅ 訂閱成功 (ID: 1)

✅ MQTT 訂閱者已就緒，等待訊息...
💡 提示：
   1. 此訂閱者會持續接收訊息
   2. 可以執行 lesson6_1.ipynb 來發布測試訊息
   3. 執行 Cell 5 查看統計資訊
   4. 執行 Cell 6 關閉連線



📩 收到訊息 #1
   主題: 客廳/感測器
   時間: 2025-12-13 01:47:23
   🌡️  溫度: 19.28°C
   💧 濕度: 77.51%
   💡 電燈: 關
   📱 裝置: Jupyter 測試裝置
   ⏰ 發送時間: 2025-12-13T01:47:23.043203


In [7]:
# Cell 5: 查看訊息統計和歷史記錄（可重複執行）

print("=" * 60)
print("📊 MQTT 訊息統計")
print("=" * 60)
print(f"📬 總接收訊息數: {message_count}")
print(f"📝 歷史記錄筆數: {len(message_history)}")
print(f"🔌 連線狀態: {'✅ 已連線' if connected else '❌ 未連線'}")
print("=" * 60)

if message_history:
    print("\n📋 最近的訊息記錄：")
    print("=" * 60)
    
    # 顯示最近 10 筆訊息
    recent_messages = message_history[-10:]
    
    for msg in recent_messages:
        print(f"\n訊息 #{msg['count']}:")
        print(f"  主題: {msg['topic']}")
        print(f"  接收時間: {msg['received_at']}")
        
        if isinstance(msg['data'], dict):
            if 'temperature' in msg['data']:
                print(f"  溫度: {msg['data']['temperature']}°C")
            if 'humidity' in msg['data']:
                print(f"  濕度: {msg['data']['humidity']}%")
            if 'light_status' in msg['data']:
                print(f"  電燈: {msg['data']['light_status']}")
        
        print("  " + "-" * 56)
    
    # 統計分析
    if any('temperature' in msg['data'] for msg in message_history if isinstance(msg['data'], dict)):
        temps = [msg['data']['temperature'] for msg in message_history 
                 if isinstance(msg['data'], dict) and 'temperature' in msg['data']]
        if temps:
            print(f"\n📈 溫度統計：")
            print(f"  平均: {sum(temps)/len(temps):.2f}°C")
            print(f"  最高: {max(temps):.2f}°C")
            print(f"  最低: {min(temps):.2f}°C")
    
    if any('humidity' in msg['data'] for msg in message_history if isinstance(msg['data'], dict)):
        humidities = [msg['data']['humidity'] for msg in message_history 
                      if isinstance(msg['data'], dict) and 'humidity' in msg['data']]
        if humidities:
            print(f"\n💧 濕度統計：")
            print(f"  平均: {sum(humidities)/len(humidities):.2f}%")
            print(f"  最高: {max(humidities):.2f}%")
            print(f"  最低: {min(humidities):.2f}%")
    
    # 電燈狀態統計
    if any('light_status' in msg['data'] for msg in message_history if isinstance(msg['data'], dict)):
        light_on = sum(1 for msg in message_history 
                      if isinstance(msg['data'], dict) and msg['data'].get('light_status') == '開')
        light_off = sum(1 for msg in message_history 
                       if isinstance(msg['data'], dict) and msg['data'].get('light_status') == '關')
        if light_on + light_off > 0:
            print(f"\n💡 電燈狀態統計：")
            print(f"  開: {light_on} 次")
            print(f"  關: {light_off} 次")
else:
    print("\n💡 尚未接收到任何訊息")
    print("   請執行 lesson6_1.ipynb 來發布測試訊息")


📊 MQTT 訊息統計
📬 總接收訊息數: 0
📝 歷史記錄筆數: 0
🔌 連線狀態: ✅ 已連線

💡 尚未接收到任何訊息
   請執行 lesson6_1.ipynb 來發布測試訊息


In [5]:
# Cell 6: 清理連線（測試結束時執行）

def cleanup():
    """清理 MQTT 連線"""
    global connected
    if 'client' in globals():
        try:
            print("🧹 正在關閉連線...")
            client.loop_stop()
            client.disconnect()
            print("✅ 連線已關閉")
            
            # 顯示最終統計
            print("\n📊 最終統計：")
            print(f"   總接收訊息數: {message_count}")
            print(f"   歷史記錄筆數: {len(message_history)}")
            
            connected = False
        except Exception as e:
            print(f"⚠️ 清理時發生錯誤: {e}")
    else:
        print("ℹ️ 沒有需要清理的連線")

cleanup()


🧹 正在關閉連線...
👋 已正常斷開連線
✅ 連線已關閉

📊 最終統計：
   總接收訊息數: 1
   歷史記錄筆數: 1
